# Introduction aux Modèles de Diffusion

Ce notebook présente les concepts fondamentaux des modèles de diffusion dans le contexte de l'intelligence artificielle générative.

## Table des matières
1. [Vérification de l'environnement](#verification)
2. [Théorie des modèles de diffusion](#theorie)
3. [Premier exemple avec Diffusers](#exemple)
4. [Génération d'images simples](#generation)
5. [Visualisation des résultats](#visualisation)

## 1. Vérification de l'environnement {#verification}

Commençons par vérifier que toutes les dépendances nécessaires sont installées et fonctionnelles.

In [1]:
# Import des bibliothèques essentielles
import torch
import numpy as np
import matplotlib.pyplot as plt
from diffusers import StableDiffusionPipeline
from transformers import CLIPTokenizer
import warnings
warnings.filterwarnings('ignore')

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU détecté: {torch.cuda.get_device_name(0)}")
else:
    print("Utilisation du CPU")

PyTorch version: 2.8.0+cu128
CUDA disponible: False
Utilisation du CPU


## 2. Théorie des modèles de diffusion {#theorie}

Les modèles de diffusion sont une classe de modèles génératifs qui apprennent à générer des données en simulant un processus de diffusion inverse.

### Processus de diffusion directe
Le processus directe ajoute progressivement du bruit gaussien à une image jusqu'à obtenir du bruit pur.

### Processus de diffusion inverse
Le modèle apprend à inverser ce processus, en partant du bruit pour reconstruire une image cohérente.

In [ ]:
# Simulation simple du processus de diffusion
def simulate_diffusion_process(image, num_steps=1000):
    """
    Simule le processus de diffusion directe en ajoutant du bruit progressivement
    """
    noisy_images = []
    current_image = image.copy()
    
    for t in range(0, num_steps, num_steps//10):
        # Calcul du niveau de bruit basé sur un planning linéaire
        noise_level = t / num_steps
        noise = np.random.normal(0, 1, image.shape)
        
        # Ajout du bruit à l'image
        noisy_image = (1 - noise_level) * image + noise_level * noise
        noisy_images.append(noisy_image)
    
    return noisy_images

# Création d'une image simple pour la démonstration
original_image = np.zeros((64, 64))
original_image[20:44, 20:44] = 1.0  # Carré blanc

# Simulation du processus de diffusion
noisy_sequence = simulate_diffusion_process(original_image)

# Visualisation
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
for i, img in enumerate(noisy_sequence):
    row = i // 5
    col = i % 5
    axes[row, col].imshow(img, cmap='gray')
    axes[row, col].set_title(f'Étape {i+1}')
    axes[row, col].axis('off')

plt.suptitle('Processus de diffusion directe')
plt.tight_layout()
plt.show()

## 3. Premier exemple avec Diffusers {#exemple}

Utilisons maintenant la bibliothèque Diffusers pour charger un modèle pré-entraîné.

In [ ]:
# Configuration du modèle
model_id = "runwayml/stable-diffusion-v1-5"
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Chargement du modèle {model_id} sur {device}...")
print("Cela peut prendre quelques minutes lors du premier lancement.")

# Chargement du pipeline
try:
    pipe = StableDiffusionPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        variant="fp16" if device == "cuda" else None
    )
    pipe = pipe.to(device)
    print("Modèle chargé avec succès !")
except Exception as e:
    print(f"Erreur lors du chargement du modèle: {e}")
    print("Vous pourriez avoir besoin d'une connexion internet pour télécharger le modèle.")

## 4. Génération d'images simples {#generation}

Maintenant, générons quelques images à partir de descriptions textuelles.

In [ ]:
# Prompts de test
prompts = [
    "a beautiful sunset over mountains",
    "a cute cat sitting on a book",
    "a futuristic city with flying cars",
    "a peaceful garden with flowers"
]

# Génération d'images
generated_images = []

for prompt in prompts:
    print(f"Génération pour: '{prompt}'")
    try:
        # Génération avec paramètres optimisés pour la rapidité
        image = pipe(
            prompt,
            num_inference_steps=20,  # Réduction du nombre d'étapes pour la rapidité
            guidance_scale=7.5,      # Contrôle de l'adhésion au prompt
            height=512,
            width=512
        ).images[0]
        
        generated_images.append((prompt, image))
        print("✓ Génération réussie")
        
    except Exception as e:
        print(f"✗ Erreur lors de la génération: {e}")
        generated_images.append((prompt, None))

## 5. Visualisation des résultats {#visualisation}

Affichons les images générées avec leurs prompts correspondants.

In [ ]:
# Visualisation des résultats
num_images = len([img for prompt, img in generated_images if img is not None])

if num_images > 0:
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    axes = axes.flatten()
    
    idx = 0
    for prompt, image in generated_images:
        if image is not None and idx < 4:
            axes[idx].imshow(image)
            axes[idx].set_title(f"'{prompt}'", fontsize=10, wrap=True)
            axes[idx].axis('off')
            idx += 1
    
    # Masquer les axes non utilisés
    for i in range(idx, 4):
        axes[i].axis('off')
    
    plt.suptitle('Images générées par le modèle de diffusion', fontsize=16)
    plt.tight_layout()
    plt.show()
else:
    print("Aucune image n'a pu être générée. Vérifiez votre configuration.")

## Conclusion

Dans ce notebook d'introduction, nous avons :

1. ✅ Vérifié l'installation des dépendances
2. ✅ Exploré la théorie des modèles de diffusion
3. ✅ Simulé le processus de diffusion directe
4. ✅ Chargé un modèle Stable Diffusion pré-entraîné
5. ✅ Généré des images à partir de descriptions textuelles

### Prochaines étapes

Dans les prochains notebooks, nous approfondirons :
- L'architecture détaillée des modèles de diffusion
- Les techniques d'optimisation et de fine-tuning
- L'utilisation avancée des prompts
- L'intégration avec d'autres modèles génératifs

### Ressources supplémentaires

- [Documentation Diffusers](https://huggingface.co/docs/diffusers/)
- [Papier original DDPM](https://arxiv.org/abs/2006.11239)
- [Stable Diffusion](https://arxiv.org/abs/2112.10752)